# edit method

In [1]:
import numpy as np

# 解析 TSP 文件
def parse_tsp(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        cities = []
        for line in lines:
            parts = line.split()
            if len(parts) == 3 and parts[0].isdigit():
                cities.append((float(parts[1]), float(parts[2])))
        return cities

# 计算两点间距离
def distance(city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# 计算路径长度
def path_length(cities, tour):
    total_distance = 0
    for i in range(len(tour)):
        total_distance += distance(cities[tour[i-1]], cities[tour[i]])
    return total_distance

# 解析 .opt.tour 文件，以比较最优路径长度
def parse_opt_tour(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        opt_tour = []
        for line in lines:
            parts = line.split()
            if parts[0].isdigit():
                opt_tour.append(int(parts[0]) - 1)  # 减1是因为TSP文件通常从1开始编号，而Python列表从0开始
        return opt_tour

# 根据 .opt.tour 文件计算路径长度
def calculate_opt_length(cities, opt_tour):
    total_distance = 0
    for i in range(len(opt_tour)):
        total_distance += distance(cities[opt_tour[i - 1]], cities[opt_tour[i]])
    return total_distance


# 解析 eli51.tsp 和 eli51.opt.tour 文件
cities = parse_tsp('eil51.tsp.txt')
opt_tour = parse_opt_tour('eil51.opt.tour.txt')
opt_length = calculate_opt_length(cities, opt_tour)



In [6]:
# 2-opt策略的模拟退火算法
def simulated_annealing_2_opt(cities, initial_temp, final_temp, alpha):
    current_temp = initial_temp
    current_tour = np.random.permutation(len(cities))
    current_length = path_length(cities, current_tour)
    
    while current_temp > final_temp:
        i, j = np.sort(np.random.choice(len(cities), 2, replace=False))
        new_tour = two_opt_swap(current_tour, i, j)
        new_length = path_length(cities, new_tour)
        
        if new_length < current_length or np.random.rand() < np.exp((current_length - new_length) / current_temp):
            current_tour = new_tour
            current_length = new_length
            
        current_temp *= alpha
    
    return current_tour, current_length

# 移动城市策略的模拟退火算法
def simulated_annealing_move_city(cities, initial_temp, final_temp, alpha):
    current_temp = initial_temp
    current_tour = np.random.permutation(len(cities))
    current_length = path_length(cities, current_tour)
    
    while current_temp > final_temp:
        city_index = np.random.randint(len(cities))
        new_position = np.random.randint(len(cities))
        new_tour = move_city(current_tour, city_index, new_position)
        new_length = path_length(cities, new_tour)
        
        if new_length < current_length or np.random.rand() < np.exp((current_length - new_length) / current_temp):
            current_tour = new_tour
            current_length = new_length
            
        current_temp *= alpha
    
    return current_tour, current_length

# 运行模拟退火算法多次
def multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, strategy):
    best_tour = None
    best_length = float('inf')
    
    for _ in range(runs):
        if strategy == '2-opt':
            tour, length = simulated_annealing_2_opt(cities, initial_temp, final_temp, alpha)
        elif strategy == 'move_city':
            tour, length = simulated_annealing_move_city(cities, initial_temp, final_temp, alpha)
        
        if length < best_length:
            best_length = length
            best_tour = tour
    
    return best_tour, best_length

# 设置参数
runs = 500  # 运行次数
initial_temp = 15000  # 初始温度
final_temp = 0.001    # 最终温度
alpha = 0.999         # 冷却率

# 解析文件
cities = parse_tsp('eil51.tsp.txt')
opt_tour = parse_opt_tour('eil51.opt.tour.txt')
opt_length = calculate_opt_length(cities, opt_tour)

# 运行模拟退火算法多次，使用2-opt
best_tour_2opt, best_length_2opt = multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, '2-opt')

# 运行模拟退火算法多次，使用移动城市策略
best_tour_move_city, best_length_move_city = multiple_runs_simulated_annealing(cities, runs, initial_temp, final_temp, alpha, 'move_city')

print("使用2-opt找到的最佳路径长度:", best_length_2opt)
print("使用移动城市策略找到的最佳路径长度:", best_length_move_city)
print("最优路径长度:", opt_length)

使用2-opt找到的最佳路径长度: 431.4330821323561
使用移动城市策略找到的最佳路径长度: 437.0980811195647
最优路径长度: 429.983311983384
